In [99]:
from arch.unitroot import ADF, PhillipsPerron
import pandas as pd
import statsmodels.api as sm
import yfinance as yf
import numpy as np
import statsmodels.api as sm
import scipy.stats as stats

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


michelin = yf.Ticker("MC.PA")
michelin = michelin.history(start="2022-10-03", end="2023-09-22")
log_close = np.log(michelin["Close"])


def racine_unitaire(data):
    """
    Analyse et teste la présence de racine unitaire sur la série temporelle 'data'.
    
    Parameters:
    - data (array-like): La série temporelle à analyser.
    
    Returns:
    None
    """
    n = len(data)

    # Estimation du modèle 3
    F3, F2, pp3, b = model3(data, n)

    d_data = np.diff(data)

    # Test de racine unitaire et hypothèse jointe H03
    if pp3 >= 0.05 and F3 <= 6.25:
        print("✔️ H0 \n✔️ H03")

        # Test d'hypothèse jointe H02
        if F2 <= 4.68: 
            # Estimation du modèle 2
            F1, pp2, c = model2(data, n)
            print("✔️ H02")

            # Test de racine unitaire et hypothèse jointe H01
            if pp2 >= 0.05 and F1 <= 4.59:
                print("✔️ H0 \n✔️ H01")
                # Test de moyenne
                res = stats.ttest_1samp(data, popmean=0)
                
                if res.pvalue >= 0.05:
                    # Estimation du modele 1
                    pp1 = model1(data, n)
                    
                    # Test de racine unitaire du modele 1
                    if pp3 >= 0.05:
                        print("Processus AR(1) x_t = phi_1 x_t-& + a_t")
                    else:
                        print("Processus DS, sans dérive")
                else:
                    print("Processus DS, sans dérive")
            else:
                if c >= 0.05:
                    print("Processus AR(1) x_t = c + phi_1 x_t-& + a_t")
                else:
                    print("Processus AR(1) x_t = phi_1 x_t-& + a_t")
        else:
            # Estimation du modèle différence premières
            print("❌ H02")
            t = np.arange(len(d_data))
            t = sm.add_constant(t)

            model = sm.OLS(d_data, t).fit()

            # Test de la significativité de la pente du modèle
            if model.pvalues[0] >= 0.05:
                print("Processus DS avec dérive")
            # Test de la significativité de la constante
            else:
                if model.pvalues[1] >= 0.05:
                    print("Processus TS, delta_x suit une tendance linéaire sans constante")
                else:
                    print("Processus TS, delta_x suit une tendance linéaire sans constante")
    else:
        print("pas ok")



def model3(data, n):
    # Test de Phillips Perron cst tendance
    pp = PhillipsPerron(data, trend="ct")


    # Passage aux différences premières
    d_data = np.diff(data)
    # Calcul du modèle 3 contraint
    cst = sm.add_constant(np.zeros(d_data.shape[0]))
    model = sm.OLS(d_data, cst).fit()

    # Calcul des scr du modele 3 contraint
    scr3c = np.sum(model.resid**2)

    # Calcul des scr du modèle 3
    scr3 = np.sum(pp.regression.resid**2)

    # Calcul des scr contraint
    scrc = np.sum(d_data**2)

    # Calcul de la statistique de Fisher
    F3 = ((scr3c - scr3)/2)/((scr3)/(n-3))
    F2 = ((scrc - scr3)/3)/((scr3)/(n-3))

    return F3, F2, pp.pvalue, pp.regression.pvalues['trend']

def model2(data, n):
    # Test de Phillips Perron constante 
    pp = PhillipsPerron(data, trend="c")

    # Passage aux différences premières
    d_data = np.diff(data)

    # Calcul des scr du modèle 2
    scr2 = np.sum(pp.regression.resid**2)

    # Calcul des scr contraint à H01
    scrc = np.sum(d_data**2)

    # Calcul de la statistique de Fisher
    F1 = ((scrc - scr2)/2)/((scr2)/(n-2))
    return F1, pp.pvalue, pp.regression.pvalues["const"]

def model1(data, n):
    # Test de Phillips Perron constante 
    pp = PhillipsPerron(data, trend="n")

    # Passage aux différences premières
    d_data = np.diff(data)


    return pp.value



racine_unitaire(log_close)

✔️ H0 
✔️ H03
✔️ H02
✔️ H0 
✔️ H01
Processus DS, sans dérive
